# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [2]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [12]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Library.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    if Stats == None:
        print('Stats for %s failed CPU-time %.4f' % (filename, time))
        return
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))
    
# Get R2/Q2 and constraints
def collate_stats(model, parameter, X, Y, verbose=False):  
    if verbose: print(Y.shape, parameter.Y.shape)
    RQ2 = r2_score(parameter.Y, Y, multioutput='variance_weighted')
    if verbose: print('RQ2 =', RQ2)
    X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
    Y = tf.convert_to_tensor(np.float32(Y))
    L2, _ = Loss_SV(Y, model.S)
    L2 = np.mean(L2.numpy())
    if verbose: print('Loss_SV =', L2)
    L3, _ = Loss_Vin(Y, model.Pin, X, model.mediumbound)
    L3 = np.mean(L3.numpy())
    if verbose: print('Loss_Vin =', L3)
    L = (L2+L3)/2
    if verbose: print('Constraints =', L)
    return RQ2, L

['Build_Model_RC.ipynb', 'Duplicate_Model.ipynb', 'Dataset_experimental', 'Result', '.DS_Store', 'Dataset_model', 'LICENSE', 'Reservoir', 'Build_Model_MM.ipynb', 'Build_Experimental.ipynb', 'Build_Model_ANN_Dense.ipynb', 'Build_Model_AMN.ipynb', 'environment_amn.yml', 'Build_Model_Dense.ipynb', 'Library', 'README.md', '.gitignore', 'Figures', 'Build_Dataset.ipynb', '.ipynb_checkpoints', 'Dataset_input', '.git', 'Figures.ipynb', 'Tutorial.ipynb', 'environment_amn_light.yml']


# Create and Train ANN




In [23]:
# Create, train and evaluate ANN models with FBA simulated training set for E. coli core
# To get R2 run with xfold = 0 otherwise Q2 is computed for given xflod

# What you can change
seed = 2
np.random.seed(seed=seed)
trainname = 'e_coli_core_EB_10'
xfold = 0
Maxloop = 3
# End of what you can change

# Load training set
trainingfile = DIRECTORY+'Dataset_model/'+trainname
cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
OBJ = [r.id for r in cobramodel.reactions]
parameter = TrainingSet()
parameter.load(trainingfile)
RQ2, Loss = [], []

for Nloop in range(Maxloop):

    # Set all pred fluxes to zero
    Y, zeros = {}, 0 * parameter.Y 
    for i in range(len(OBJ)):
        Y[i] = zeros[:,0] # zero vector

    # Train for all flux are taken one after another
    for i in range(len(OBJ)):
        objective = [OBJ[i]]
        print('----------------------------------------------------------------------',i+1,'/',len(OBJ), objective)
        # create model
        if np.sum(parameter.Y[:,i]) == 0:
            continue # Skip all parameter.Y beeing at zero
        model = Neural_Model(trainingfile = trainingfile,
                             objective=objective, 
                             model_type = 'ANN_Dense',
                             scaler=True,
                             n_hidden = 1, hidden_dim = 50,
                             epochs = 500, xfold = xfold)
        # Train and evaluate
        start_time = time.time()
        try:
            reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
        except:
            reservoir, pred, stats, _ = None, zeros, None, None
        # Printing cross-validation results
        delta_time = time.time() - start_time
        printout(objective, stats, reservoir, delta_time)
        Y[i] = pred[:,0]

    # Collate all predicted Y and get stats and constraints
    Y = np.transpose(np.asarray(list(Y.values())))
    rq2, l = collate_stats(model, parameter, X, Y, verbose=True)
    RQ2.append(rq2)
    Loss.append(l)

# Print stats averaged over all iterations
rqt = 'R2' if xfold < 2 else 'Q2'
print(rqt, '= %.4f (+/- %.4f) Loss = %.4f (+/- %.4f)' \
      % (np.mean(RQ2), np.std(RQ2), np.mean(Loss), np.std(Loss)))

---------------------------------------------------------------------- 1 / 154 ['PFK']
nbr parameters: 1101
1/1 [==============================] - 0s 6ms/step
train = 0.81 test = 0.81 loss-train = -1.000000 loss-test = -1.000000 iter=0
Stats for ['PFK'] CPU-time 4.1012
R2 = 0.8075 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
Q2 = 0.8075 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
---------------------------------------------------------------------- 2 / 154 ['PFL']
nbr parameters: 1101
1/1 [==============================] - 0s 7ms/step
train = 0.64 test = 0.64 loss-train = -1.000000 loss-test = -1.000000 iter=0
Stats for ['PFL'] CPU-time 4.3745
R2 = 0.6351 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
Q2 = 0.6351 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
---------------------------------------------------------------------- 3 / 154 ['PGI_for']
nbr parameters: 1101
1/1 [==============================] - 0s 17ms/step
looping bad training iter=0 r2=-536.9346
1/1 [==============